In [ ]:
%load_ext jupyter_black

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import pulp as pl

from copy import deepcopy

# Creates a random valuated graph

In [ ]:
def get_graphs(min_capacity, max_capacity, n_nodes, n_edges):
    graph = nx.gnm_random_graph(n_nodes, n_edges, directed=True)
    dual_graph = deepcopy(graph)

    for (u, v) in graph.edges():
        graph.edges[u, v]["capacity"] = np.random.uniform(min_capacity, max_capacity)

    return graph, dual_graph

# Column Generation

In [ ]:
def get_n_paths(graph, source, destination, max_iter=16):
    """Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination"""
    list_ = []
    for (index, path) in enumerate(
        nx.all_simple_paths(graph, source, destination, cutoff=10)
    ):
        list_.append(path)
        if index >= max_iter - 1:
            return list_
    return list_

## Master

### Initialise the master problem

We consider a network with:
- $\mathcal{V}$, a set of nodes
- $\mathcal{E}$, a set of directed edges with capacities $C_e, e\in \mathcal{E}$
- $d$, a connection/demand, with predefined path set $\mathcal{P}_d$ between nodes $s_d$ and $t_d$
- Any path $p$ is represented by a set of edges, i.e. $p\subseteq \mathcal{E}$

We denote by $x_p$ the flow allocated to a path $p\in \mathcal{P}_d$, and $\pi_e$ the associated dual variables.

We wish to find feasible flows for each path that maximizes the sum of all flows $\sum_{p\in \mathcal{P}_d} x_p$, i.e. the maximum flow problem formulated through path variables instead of flow variables on edges.

$$
\begin{align}
\max & \sum_{p\in \mathcal{P}_d}\\
\text{s.t.} & \sum_{p\in \mathcal{P}_d, e\in p} x_p \leq C_e,\quad \forall e\in \mathcal{E}\, (\pi_e)\\
& x_p \geq 0,\quad p\in \mathcal{P}_d
\end{align}
$$

For any given path $q\in D$, with $D$ the columns out of the optimal basis $B$, we wish to maximize the reduced cost 
$$
1 - \sum_{e\in q} \pi_e,
$$
i.e., minimizing $\sum_{e\in q}$, that is finding the shortest path $q$ weighted with $\pi$.

N.B.: The problem of maximum flow is equivalent to maximum path generation **if we consider all possible paths**

In this way, we can iteratively add paths to our initial set $\mathcal{P}_d$ until the solution given by the master problem is optimal, i.e. the reduced cost is negative:

1. Initialise the master problem (MP) with a set $\mathcal{P}_d$ containing paths from $s_d$ to $t_d$
2. Solve (MP) and note the obtained dual variables $\pi$
3. Compute the shortest path $q$ from $s_d$ to $t_d$
4. Compute the reduced cost $r_q=1 - \sum_{e\in q}\pi_e$
    1. If $r_q \geq 0$, then add $q$ to $\mathcal{P}_d$, and continue from 2)
    2. Otherwise, stop. The solution is optimal

In [ ]:
def add_paths(graph, max_capacity, lpProb, paths, path_count, path2idx):
    """Add paths in lpProb (new variables Xp and new constraints or terms in constraints Ce)"""

    # For every path add one variable, and its constraints
    for path in paths:

        # add a variable
        prov = pl.LpVariable(
            name="X" + str(path_count),
            lowBound=0,
            upBound=max_capacity,
            cat=pl.LpContinuous,
        )
        lpProb.objective.addterm(prov, 1)

        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path) - 1):
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1)  # add term in constraint
            else:
                lpProb.constraints[index_ctr] = (
                    prov <= graph.edges[(path[index], path[index + 1])]
                )
                ctr_lists.append((path[index], path[index + 1]))

        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

### Dual problem

In [ ]:
def get_duals(lpPb):
    """Get the dual variables from a problem Pulp"""
    return [c.pi for _, c in list(lpPb.constraints.items())]

### Graph Params

In [ ]:
min_capacity = 5
max_capacity = 15
density = 0.05
n_nodes = 5 * 10**1
n_edges = density * (n_nodes * (n_nodes - 1))
source, destination = 0, 1  # don't need to randomize (graph is randomized)
density, n_nodes, n_edges

### Iterative procedure

1. Density is the proportion of edges in a full connected graph. A dense graph has $n(n-1)$ edges where $n$ is the number of nodes.
2. `maximum_flow` from networkx has a linear trend w.r.t. the graph density.
3. Column generation has an exponential trend w.r.t. density (keep a density < 0.3 !!!)

In [ ]:
def modify_weights_graph(graph, ctr_lists, new_weights):
    """Utility function for setting weights (dual problem of shortest path)"""
    nx.set_edge_attributes(graph, 0, name="weight")

    nx.set_edge_attributes(
        graph,
        values={edge: dual for edge, dual in zip(ctr_lists, new_weights)},
        name="weight",
    )

In [ ]:
lpProb = pl.LpProblem(
    name="path_generation",
    sense=pl.LpMaximize,
)
lpProb += 0  # initiate the objective function to 0 (necessary line of code)

path2idx = (
    {}
)  # useful to know the final paths at the end and calculate the shortest path for dual problem
ctr_lists = []
path_count = 0
graph, dual_graph = get_graphs(min_capacity, max_capacity, n_nodes, n_edges)

path_count, path2idx = add_paths(
    graph=graph,
    lpProb=lpProb,
    max_capacity=max_capacity,
    paths=get_n_paths(graph, source, destination, max_iter=1),
    path_count=path_count,
    path2idx=path2idx,
)
solver = pl.PULP_CBC_CMD(msg=False)
result = lpProb.solve(solver)

In [ ]:
go_on = True
reduced_cost_arr = np.array([])
reduced_cost = 1

while go_on:
    # search of path q (dual problem)
    duals = get_duals(lpProb)
    modify_weights_graph(dual_graph, ctr_lists, duals)
    path_q = nx.shortest_path(dual_graph, source, destination, weight="weight")

    # compute reduced cost for dual

    for e in range(len(path_q) - 1):
        try:
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError:  # <|-- constraint does not exist (so dual == 0)

            pass

    # stop procedure ?
    if reduced_cost > 0:  # add path q in lp Problem
        reduced_cost_arr = np.append(reduced_cost_arr, reduced_cost)
        path_count, path2idx = add_paths(
            graph=graph,
            lpProb=lpProb,
            max_capacity=max_capacity,
            paths=[path_q],
            path_count=path_count,
            path2idx=path2idx,
        )
        lpProb.solve(solver)
    else:
        go_on = False

In [ ]:
reduced_cost_arr

In [ ]:
flow = 0
for var in lpProb.variables():
    flow += var.value()
print(flow)

In [ ]:
paths = {
    path2idx[int(str(var)[1:])]: var.value()
    for var in lpProb.variables()
    if var.value() > 0
}
paths

In [ ]:
nx.maximum_flow(graph, 0, 1)